# TensorBoard Debugger notebook
---

<font color='red'> <h3>Tested with TensorFlow 1.14</h3></font>

This notebook introduces how to connect your TensorFlow application the TensorBoard debugger.

<p>
<h1>Machine Learning on <a href="https://github.com/logicalclocks/hopsworks">Hopsworks
</a></h1> 
</p>

![hops.png](../../images/hops.png)

## The `hops` python module

`hops` is a helper library for Hops that facilitates development by hiding the complexity of running applications and iteracting with services.

Have a feature request or encountered an issue? Please let us know on <a href="https://github.com/logicalclocks/hops-util-py">github</a>.

### Using the `experiment` module

To be able to run your Machine Learning code in Hopsworks, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function.

The `experiment` module provides an api to Python programs such as TensorFlow, Keras and PyTorch on a Hopsworks on any number of machines and GPUs.

An Experiment could be a single Python program, which we refer to as an **Experiment**. 

Grid search or genetic hyperparameter optimization such as differential evolution which runs several Experiments in parallel, which we refer to as **Parallel Experiment**. 

ParameterServerStrategy, CollectiveAllReduceStrategy and MultiworkerMirroredStrategy making multi-machine/multi-gpu training as simple as invoking a function for orchestration. This mode is referred to as **Distributed Training**.

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project's Experiments folder.

The directory could in practice be used to store other data that should be accessible after the experiment is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```

### Using the `hdfs` module
The `hdfs` module provides a method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`

```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

```python
# Downloading the mnist dataset to the current working directory
from hops import hdfs
mnist_hdfs_path = hdfs.project_path() + "Resources/mnist"
local_mnist_path = hdfs.copy_to_local(mnist_hdfs_path)
```

### Documentation
See the following links to learn more about running experiments in Hopsworks

- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/experiment.html">Learn more about experiments</a>
<br>
- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/hopsML.html">Building End-To-End pipelines</a>
<br>
- Give us a star, create an issue or a feature request on  <a href="https://github.com/logicalclocks/hopsworks">Hopsworks github</a>

### Managing experiments
Experiments service provides a unified view of all the experiments run using the `experiment` module.
<br>
As demonstrated in the gif it provides general information about the experiment and the resulting metric. Experiments can be visualized meanwhile or after training in a TensorBoard.
<br>
<br>
![Image7-Monitor.png](../../images/experiments.gif)

In [ ]:
def debugger_example():
    # Copyright 2016 The TensorFlow Authors. All Rights Reserved.
    #
    # Licensed under the Apache License, Version 2.0 (the "License");
    # you may not use this file except in compliance with the License.
    # You may obtain a copy of the License at
    #
    #     http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific language governing permissions and
    # limitations under the License.
    # ==============================================================================
    """Demo of the tfdbg curses CLI: Locating the source of bad numerical values.

    The neural network in this demo is larged based on the tutorial at:
      tensorflow/examples/tutorials/mnist/mnist_with_summaries.py

    But modifications are made so that problematic numerical values (infs and nans)
    appear in nodes of the graph during training.
    """

    import argparse
    import sys
    import os

    import tensorflow as tf

    from tensorflow.examples.tutorials.mnist import input_data
    from tensorflow.python import debug as tf_debug
    
    from hops import tensorboard

    IMAGE_SIZE = 28
    HIDDEN_SIZE = 500
    NUM_LABELS = 10
    RAND_SEED = 42
    LEARNING_RATE = 0.025
    STEPS = 10
    
    tensorboard_debug_address = tensorboard.interactive_debugger()
    
    # How you easily get the TensorBoard logdir for summaries
    tensorboard_logdir = tensorboard.logdir()

    # Import data
    mnist = input_data.read_data_sets(os.getcwd(),
                                        one_hot=True,
                                        fake_data=False)

    def feed_dict(train):
        if train:
            xs, ys = mnist.train.images, mnist.train.labels
        else: 
            xs, ys = mnist.test.images, mnist.test.labels
            
        return {x: xs, y_: ys}

    sess = tf.InteractiveSession()

    # Create the MNIST neural network graph.

    # Input placeholders.
    with tf.name_scope("input"):
      x = tf.placeholder(
          tf.float32, [None, IMAGE_SIZE * IMAGE_SIZE], name="x-input")
      y_ = tf.placeholder(tf.float32, [None, NUM_LABELS], name="y-input")

    def weight_variable(shape):
      """Create a weight variable with appropriate initialization."""
      initial = tf.truncated_normal(shape, stddev=0.1, seed=RAND_SEED)
      return tf.Variable(initial)

    def bias_variable(shape):
      """Create a bias variable with appropriate initialization."""
      initial = tf.constant(0.1, shape=shape)
      return tf.Variable(initial)

    def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
      """Reusable code for making a simple neural net layer."""
      # Adding a name scope ensures logical grouping of the layers in the graph.
      with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope("weights"):
          weights = weight_variable([input_dim, output_dim])
        with tf.name_scope("biases"):
          biases = bias_variable([output_dim])
        with tf.name_scope("Wx_plus_b"):
          preactivate = tf.matmul(input_tensor, weights) + biases

        activations = act(preactivate)
        return activations

    hidden = nn_layer(x, IMAGE_SIZE**2, HIDDEN_SIZE, "hidden")
    logits = nn_layer(hidden, HIDDEN_SIZE, NUM_LABELS, "output", tf.identity)
    y = tf.nn.softmax(logits)

    with tf.name_scope("cross_entropy"):

      diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits)
      with tf.name_scope("total"):
        cross_entropy = tf.reduce_mean(diff)

    with tf.name_scope("train"):
      train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(
          cross_entropy)

    with tf.name_scope("accuracy"):
      with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
      with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    sess.run(tf.global_variables_initializer())
    sess = tf_debug.TensorBoardDebugWrapperSession(
            sess, tensorboard_debug_address)

    # Add this point, sess is a debug wrapper around the actual Session if
    for i in range(STEPS):
      
      acc = sess.run(accuracy, feed_dict=feed_dict(False))
      print("Accuracy at step %d: %s" % (i, acc))

      sess.run(train_step, feed_dict=feed_dict(True))




In [ ]:
from hops import experiment
experiment.launch(debugger_example, name='tensorboard debugger', local_logdir=True)